<a href="https://colab.research.google.com/github/WandersAimless/Desktop-Python/blob/main/MarketDataFetch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Data Collection script

from ib_insync import *
import psycopg2
import datetime
import time

# Initialize IBKR connection
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=123)  # Adjust IP and client ID if necessary

# TimescaleDB Connection
def get_db_connection():
    return psycopg2.connect(
        host="localhost",  # Adjust as needed
        database="your_database",
        user="your_user",
        password="your_password"
    )

# Pull Historical Data (10 years) including Level 2 Data
def pull_historical_data(symbol, start_date, end_date):
    contract = Stock(symbol, 'SMART', 'USD')

    # Pull historical OHLC data with 5-minute intervals (you can adjust the interval)
    historical_data = ib.reqHistoricalData(
        contract,
        endDateTime=end_date,
        durationStr="10 Y",
        barSizeSetting="5 mins",
        whatToShow="MIDPOINT",
        useRTH=True,
        formatDate=1
    )

    # Retrieve Level 2 data (market depth)
    ib.reqMktDepth(contract, 5)  # Get top 5 levels of market depth

    return historical_data

# Save data to TimescaleDB
def save_to_timescaledb(symbol, historical_data, level2_data):
    conn = get_db_connection()
    cur = conn.cursor()

    # Insert OHLC data
    for bar in historical_data:
        cur.execute("""
            INSERT INTO market_data (symbol, datetime, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (symbol, datetime) DO NOTHING;
        """, (symbol, bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume))

    # Insert Level 2 data
    for i, data in enumerate(level2_data):
        cur.execute("""
            INSERT INTO market_depth (symbol, datetime, bid_price, bid_size, ask_price, ask_size, level)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (symbol, datetime, level) DO NOTHING;
        """, (symbol, datetime.datetime.now(), data.bidPrice, data.bidSize, data.askPrice, data.askSize, i))

    conn.commit()
    cur.close()
    conn.close()

# Check for gaps in the data
def check_for_gaps(symbol, start_date, end_date):
    conn = get_db_connection()
    cur = conn.cursor()

    # Query the most recent date in the database for the symbol
    cur.execute("""
        SELECT MAX(datetime) FROM market_data WHERE symbol = %s;
    """, (symbol,))
    last_date = cur.fetchone()[0]

    # If there's a gap, return the start of the gap (or the latest date missing data)
    if last_date is None or last_date < end_date:
        return last_date
    return None

# Pull historical data to fill gaps
def fill_gaps(symbol, last_date):
    start_date = last_date + datetime.timedelta(minutes=5)  # Assuming 5-minute intervals
    historical_data = pull_historical_data(symbol, start_date, datetime.datetime.now())

    if historical_data:
        save_to_timescaledb(symbol, historical_data, [])
        print(f"Filled gap for {symbol} from {start_date} to {datetime.datetime.now()}")
    else:
        print(f"No data to fill gap for {symbol} from {start_date}.")

# Real-Time Data Stream (Update continuously)
def stream_real_time_data(symbol):
    contract = Stock(symbol, 'SMART', 'USD')

    # Use reqMktData for real-time data
    ib.reqMktData(contract, "", False, False)

    # You can implement a callback function to process the live data updates
    @ib.on('tickPrice')
    def on_tick_price(tickPrice):
        print(f"Live update for {symbol}: {tickPrice}")

    # Run the event loop to keep the connection open
    ib.run()

# Main Function to Collect Data, Check Gaps, and Fill Gaps
def main(symbol):
    # Step 1: Pull 10 years of historical data (for the first time)
    end_date = datetime.datetime.now()
    historical_data = pull_historical_data(symbol, datetime.datetime(2013, 1, 1), end_date)
    level2_data = []  # Retrieve Level 2 data here as well (if needed)

    # Step 2: Save the data to TimescaleDB
    save_to_timescaledb(symbol, historical_data, level2_data)

    # Step 3: Start real-time data stream
    stream_real_time_data(symbol)

    # Step 4: Check for gaps and fill them nightly
    while True:
        gap_start_date = check_for_gaps(symbol, datetime.datetime(2013, 1, 1), end_date)
        if gap_start_date:
            fill_gaps(symbol, gap_start_date)

        # Sleep for the night before checking again
        time.sleep(86400)  # Sleep for 24 hours

# Run the script for a specific stock ticker
if __name__ == "__main__":
    main("AAPL")  # Replace with your desired stock ticker




Key Notes:
Historical Data: The script pulls 10 years of 5-minute interval data (adjustable) and includes level 2 data (market depth). You can modify the data frequency as needed (e.g., 1 min, 5 min).
Saving to TimescaleDB: Data is inserted into TimescaleDB tables (market_data for OHLC data and market_depth for level 2 data). Make sure these tables are already created in your TimescaleDB database.
Gap Detection: The check_for_gaps() function queries the latest data in the database to identify any gaps. If any are found, it will fill them by pulling historical data.
Real-Time Data Stream: The stream_real_time_data() function initiates a live data stream for a given ticker. The tickPrice event listener will capture real-time price updates and can be used to feed the predictions to your machine learning model.
Nightly Gap Filling: The script continuously checks for missing data and attempts to fill any gaps at night (or any interval you configure).

In [1]:
#TimescaleDB Database Setup
#For individual tickers
# This is written for SQL not Python

CREATE TABLE market_data (
    symbol TEXT,
    datetime TIMESTAMPTZ,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume INT,
    PRIMARY KEY (symbol, datetime)
);
SELECT create_hypertable('market_data', 'datetime');

CREATE TABLE market_depth (
    symbol TEXT,
    datetime TIMESTAMPTZ,
    bid_price FLOAT,
    bid_size INT,
    ask_price FLOAT,
    ask_size INT,
    level INT,
    PRIMARY KEY (symbol, datetime, level)
);
SELECT create_hypertable('market_depth', 'datetime');
